In [ ]:
import pandas as pd
import numpy as np

path = 'D:/606/RoadMaintenance/FAF/FAF4.5.1_csv_2013-2018/FAF4.5.1_'
dest = 'D:/606/RoadMaintenance/Datasets/WeightedFAF/WeightedAvg'

startYear = 2013
endYear = 2018

for year in range(startYear, endYear+1, 1):
    df = pd.read_csv(path + f'{year}.csv')

    Truck_df = df[df['dms_mode'] == 1]

    CFS_Area = Truck_df[((Truck_df['dms_orig'] == 11) & (Truck_df['dms_dest'] == 11)) | ((Truck_df['dms_orig'] == 12) & (Truck_df['dms_dest'] == 12))]

    CFS_Area = CFS_Area.drop(columns=['fr_orig', 'fr_dest', 'fr_inmode', 'fr_outmode', 'sctg2', 'trade_type', 'dms_mode'])

    #CFS_Area = CFS_Area.rename(columns={f'tons_{year}': 'tons', f'value_{year}': 'value', f'tmiles_{year}': 'tmiles', f'curval_{year}': 'curval'})

    CFS_Area = CFS_Area.reset_index(drop=True)

    def weighted_average(group):
        tons = group['tons']
        value = group['value']
        tmiles = group['tmiles']
        curval = group['curval']
        wgt_dist = group['wgt_dist']

        total_tons = tons.sum()  # Total weight for each group

        # Calculate weighted averages, handling cases where total_tons is zero
        weighted_tons = np.where(total_tons == 0, 0, (tons * tons).sum() / total_tons)  # Weight by tons
        weighted_value = np.where(total_tons == 0, 0, (value * tons).sum() / total_tons)
        weighted_tmiles = np.where(total_tons == 0, 0, (tmiles * tons).sum() / total_tons)
        weighted_curval = np.where(total_tons == 0, 0, (curval * tons).sum() / total_tons)
        weighted_wgt_dist = np.where(total_tons == 0, 0, (wgt_dist * tons).sum() / total_tons)

        return pd.Series({
            'tons': weighted_tons,
            'value': weighted_value,
            'tmiles': weighted_tmiles,
            'curval': weighted_curval,
            'wgt_dist': weighted_wgt_dist
        })

    # Group by origin and destination and apply the weighted average function
    weighted_average = CFS_Area.groupby(['dms_orig', 'dms_dest']).apply(weighted_average).reset_index()

    weighted_average.to_csv(dest + f'{year}.csv')

"path = 'D:/606/RoadMaintenance/FAF/FAF4.5.1_csv_2013-2018/FAF4.5.1_'\ndest = 'D:/606/RoadMaintenance/Datasets/WeightedFAF/WeightedAvg'\n\nstartYear = 2013\nendYear = 2018\n\nfor year in range(startYear, endYear+1, 1):\n    df = pd.read_csv(path + f'{year}.csv')\n\n    Truck_df = df[df['dms_mode'] == 1]\n\n    CFS_Area = Truck_df[((Truck_df['dms_orig'] == 11) & (Truck_df['dms_dest'] == 11)) | ((Truck_df['dms_orig'] == 12) & (Truck_df['dms_dest'] == 12))]\n\n    CFS_Area = CFS_Area.drop(columns=['fr_orig', 'fr_dest', 'fr_inmode', 'fr_outmode', 'sctg2', 'trade_type', 'dms_mode'])\n\n    #CFS_Area = CFS_Area.rename(columns={f'tons_{year}': 'tons', f'value_{year}': 'value', f'tmiles_{year}': 'tmiles', f'curval_{year}': 'curval'})\n\n    CFS_Area = CFS_Area.reset_index(drop=True)\n\n    def weighted_average(group):\n        tons = group['tons']\n        value = group['value']\n        tmiles = group['tmiles']\n        curval = group['curval']\n        wgt_dist = group['wgt_dist']\n\n       